### Super Store Dataset

In [ ]:
# Importing the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
# Import the dataset
encoding = 'unicode_escape'
storedataset = pd.read_csv('/Users/priyankac/Downloads/Time Series/Super Store.csv', encoding = 'unicode_escape')

In [ ]:
storedataset.head()

In [ ]:
# Checking the number of rows and columns in the dataset
storedataset.shape

In [ ]:
# Checking the data types
storedataset.info()

In [ ]:
# Order date is an object type here.It needs to be converted to datetime

from datetime import datetime
storedataset['Order Date'] = pd.to_datetime(storedataset['Order Date'])

In [ ]:
storedataset.dtypes

In [ ]:
storedataset['Category'].value_counts()

In [ ]:
# From the category taking only the furniture category,here we have just one category i.e furniture. This code will
# help to exract the data if there are multiple categories
Furniture = storedataset.loc[storedataset['Category'] == 'Furniture']

# Furniture1 = storedataset.loc[storedataset['Sub-Category'] == 'Chairs'] if you want to extract sub category 
# wise chair data


In [ ]:
Furniture.head()

In [ ]:
Furniture.shape

In [ ]:
print(len(Furniture))

In [ ]:
# Checking the minimum and maximum order date for furniture
print(Furniture['Order Date'].min())
print(Furniture['Order Date'].max())

In [ ]:
# Check the columns in our Furniture dataframe
Furniture.columns

In [ ]:
cols = ['Row ID', 'Order ID',  'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Quantity', 'Discount', 'Profit']

# Check with the stakeholder what they want to forecast i.e sales or quantity or discount or profit
# I want to predict sales forecasting - 'Sales'

In [ ]:
# Drop all the above cols from Furniture dataset to retain only the 'order date' and 'sales' columns
Furniture.drop(cols , axis = 1 ,inplace = True)


In [ ]:
Furniture.shape

In [ ]:
# Sort the dataset first
Furniture.sort_values('Order Date')

In [ ]:
# Grouping the sales according to the date and getting the sum of toatl sales in a day
Furniture = Furniture.groupby('Order Date')['Sales'].sum().reset_index()

In [ ]:
Furniture

In [ ]:
# Set the 'order date' as index
Furniture = Furniture.set_index('Order Date')
Furniture.index

In [ ]:
# Our current datetime data can be tricky to work with, therefore, we will use the averages daily sales value 
# for that month instead, and we are using the start of each month as the timestamp.

y = pd.DataFrame(Furniture['Sales'].resample('MS').mean())

In [ ]:
y['2017':]

In [ ]:
# Check the trend
y.plot()

In [ ]:
# Check the dataset components by using the Decomposition method
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(y['Sales'] , period = 12)
decomposition.plot()
plt.show()

In [ ]:
#The plot above clearly shows that the sales of furniture is unstable, along with its obvious seasonality.


In [ ]:
# To check whether or data is stationary or not using the Augmented Decay Fuller Test
from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(y['Sales'])

In [ ]:
# Creating a function to explain the above values(remember this function, u can use it always)
def adf_check(timeseries):
    result = adfuller(timeseries)
    print('Augmented Decay Fuller Test : ')
    labels = ['ADF Test Statistics', 'p-value' ,'#Lags', 'No of obs' ]
    
    for a , b in zip(result, labels):
        print(b + ' : ' + str(a))
        
    if result[1] <= 0.05:
        print('\nStrong evidence against null hypothesis and my time series is stationary')
    else:
        print('\nWeak evidence against null hypothesis and my time series is non stationary')

In [ ]:
adf_check(y['Sales'])

In [ ]:
# Our data is stationary
# Trend d = 0

In [ ]:
# Calculating the D value for seasonality
y['seasonality'] = y['Sales'] - y['Sales'].shift(12)

In [ ]:
y.head()

In [ ]:
adf_check(y['seasonality'].dropna())

In [ ]:
# D = 1

In [ ]:
# To find the values of p&P and q&Q we will use graphs
from statsmodels.graphics.tsaplots import plot_pacf , plot_acf

In [ ]:
plot_pacf(y['Sales'] , lags = 14)
plt.show()

In [ ]:
# p = 0

In [ ]:
plot_acf(y['Sales'].dropna(), lags = 14)
plt.show()

In [ ]:
# q = 0

In [ ]:
plot_pacf(y['seasonality'].dropna(), lags = 14)
plt.show()

In [ ]:
plot_acf(y['seasonality'].dropna(), lags = 14)
plt.show()

In [ ]:
# P = 0
# Q = 0

In [ ]:
# Trend:
# d = 0 
# p = 0       
# q = 0

# Seasonality:
# P = 0 
# D = 1 
# Q = 0 

### Model Time Series Analysis and Forecasting

In [ ]:
# Automation to find best pdq value and less AIC value for better value
import itertools

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))


In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(y['Sales'] , order = param ,
                                 seasonal_order = param_seasonal ,
                                  enforce_stationarity=False, enforce_invertibility=False ) 
            results = model.fit()
            print('ARIMA{}x{} - AIC :{}'.format(param,param_seasonal,results.aic))
        except :
            continue

In [ ]:
# ARIMA(1, 1, 1)x(1, 1, 1, 12) - AIC :283.3661018696623

In [ ]:
# Fiting the ARIMA model
model = sm.tsa.statespace.SARIMAX(y['Sales'],
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = model.fit()


In [ ]:
print(results.summary())

In [ ]:
print(len(y))

In [ ]:
# Predict with the training dataset

y['forecast'] = results.predict(start = 37 , dynamic = True) 


In [ ]:
y.head()

In [ ]:
y.tail()

In [ ]:
y[['Sales','forecast']].plot()
plt.show()

In [ ]:
# The line plot is showing the observed values compared to the rolling forecast predictions.
# Overall, our forecasts align with the true values very well, showing an upward trend starts 
# from the beginning of the year and captured the seasonality toward the end of the year.


In [ ]:
# Will generate some future dates to make prediction

In [ ]:
from pandas.tseries.offsets import DateOffset

In [ ]:
y.head()

In [ ]:
y.tail()

In [ ]:
future_dates = [y.index[-1] + DateOffset(months = x) for x in range(0,13)] 

In [ ]:
future_dates

In [ ]:
future_dates_df = pd.DataFrame(index = future_dates[1:], columns =y.columns)

In [ ]:
future_dates_df.head()

In [ ]:
future_dates_df.tail()

In [ ]:
future_df = pd.concat([y,future_dates_df])

In [ ]:
future_df.head(20)

In [ ]:
future_df.tail(20)

In [ ]:
future_df.shape

In [ ]:
future_df['forecast'] = results.predict( start = 47, end = 59, dynamic = True)

In [ ]:
future_df[['Sales','forecast']].plot()

In [ ]:
future_df.tail(20)

In [ ]:
future_df.to_csv('sales_forecasting.csv')